# 线性回归简洁实现
## 生成数据集

In [1]:
import numpy as np
import torch
from torch.utils import data
from d2l import torch as d2l

In [2]:
true_w = torch.tensor([2,-3.4])
true_b = 4.2
features,labels = d2l.synthetic_data(true_w, true_b,1000)

## 读取数据集

In [3]:
def load_array(data_arrays,batch_size,is_train=True):
    dataset = data.TensorDataset(*data_arrays)
    return data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle=is_train)


In [4]:
batch_size = 10
data_iter = load_array((features,labels),batch_size)

In [5]:
next(iter(data_iter))

[tensor([[ 0.1932,  1.1558],
         [-0.6740,  0.9614],
         [ 0.8903, -0.5210],
         [ 0.2424,  0.0303],
         [ 1.2942,  1.3548],
         [ 1.1656, -0.2456],
         [-0.9257, -2.0221],
         [ 0.0904,  0.7035],
         [ 0.1214,  0.4985],
         [-0.7501, -0.2265]]),
 tensor([[ 0.6404],
         [-0.4175],
         [ 7.7404],
         [ 4.5853],
         [ 2.1898],
         [ 7.3557],
         [ 9.2150],
         [ 1.9927],
         [ 2.7529],
         [ 3.4575]])]

## 定义模型

In [6]:
from torch import nn
net = nn.Sequential(nn.Linear(2,1))

## 初始化模型参数

In [7]:
net[0].weight.data.normal_(0,0.01)
net[0].bias.data.fill_(0)

tensor([0.])

## 定义损失函数

In [8]:
loss = nn.MSELoss()

## 定义优化算法

In [9]:
trainer = torch.optim.SGD(net.parameters(), lr=0.03)

## 训练

In [10]:
num_epoachs = 3

In [11]:
nums_epochs = 3
for epoch in range(nums_epochs):
    for X,y in data_iter:
        l = loss(net(X),y)
        trainer.zero_grad()
        l.backward()
        trainer.step()
    l = loss(net(features),labels)
    print(f'epoch {epoch}, loss {l}:f')

epoch 0, loss 0.00030293097370304167:f
epoch 1, loss 9.988305100705475e-05:f
epoch 2, loss 9.90265907603316e-05:f


In [13]:
w = net[0].weight.data
b = net[0].bias.data
print('w的误差:',true_w-w.reshape(true_w.shape))
print('b的误差:',true_b-b)

w的误差: tensor([-0.0006,  0.0002])
b的误差: tensor([0.0003])


# 练习

1. 当将小批量的总损失（sum of losses）替换为小批量损失的平均值（average loss）时，需要如何调整学习率。



*   **原理:** 梯度下降的更新规则是 $\theta_{new} = \theta_{old} - \eta \nabla L$，其中 $\eta$ 是学习率。
    *   如果使用**总损失** $L_{sum} = \sum_{i=1}^N l_i$，则梯度为 $\nabla L_{sum} = \sum_{i=1}^N \nabla l_i$。
    *   如果使用**平均损失** $L_{avg} = \frac{1}{N} \sum_{i=1}^N l_i$，则梯度为 $\nabla L_{avg} = \frac{1}{N} \sum_{i=1}^N \nabla l_i$。
    因此，$\nabla L_{avg} = \frac{1}{N} \nabla L_{sum}$。

*   **调整方法:**
    *   如果你从使用**总损失**切换到使用**平均损失**，由于梯度的幅度减小了 $N$ 倍（其中 $N$ 是批次大小），为了保持相似的参数更新步长，你需要**提高学习率**。具体来说，新的学习率 $\eta_{new}$ 大约是旧学习率 $\eta_{old}$ 的 $N$ 倍（即 $\eta_{new} \approx N \times \eta_{old}$）。
    *   反之，如果你从使用**平均损失**切换到使用**总损失**，你需要**降低学习率**。

在实际深度学习训练中，通常使用平均损失（如 PyTorch 的 `nn.MSELoss()` 默认的 `reduction='mean'`）。这样做的好处是，学习率和正则化参数对批次大小的依赖性减小，有助于训练的稳定性。

2. 查询框架和文档,看提供了哪些损失函数和初始化方法?用Huber损失代替原损失,即$ l(y, y') = \begin{cases} \frac{1}{2\sigma}(y - y')^2 & \text{if } |y - y'| \leq \sigma \\ |y - y'| - \frac{\sigma}{2} & \text{if } |y - y'| > \sigma \end{cases} $
*   **损失函数 (Loss Functions):**
    PyTorch 的 `torch.nn` 模块提供了多种损失函数，适用于不同的任务：
    *   **回归任务:**
        *   `nn.MSELoss`: 均方误差，常用于线性回归。
        *   `nn.L1Loss`: 平均绝对误差，对异常值更鲁棒。
        *   `nn.SmoothL1Loss`: 结合了 MSE 和 L1 的优点。
        *   `nn.HuberLoss`: 对异常值更加鲁棒，在小误差时表现为 MSE，在大误差时表现为 L1。
    *   **分类任务:**
        *   `nn.CrossEntropyLoss`: 交叉熵损失，常用于多分类问题。
        *   `nn.BCEWithLogitsLoss`: 结合 Sigmoid 和 BCE loss，用于二分类问题。
        *   `nn.NLLLoss`: 负对数似然损失。

*   **初始化方法 (Initialization Methods):**
    PyTorch 的 `torch.nn.init` 模块提供了多种参数初始化策略：
    *   **常数初始化:** `nn.init.zeros_`, `nn.init.ones_`, `nn.init.constant_()`。
    *   **随机初始化:** `nn.init.normal_()`, `nn.init.uniform_()`, `nn.init.xavier_uniform_()`, `nn.init.kaiming_normal_()` 等。这些方法有助于解决梯度消失或爆炸问题，并加速模型收敛。

*   **Huber 损失:**
    你提供的 Huber 损失定义为：
    $$ l(y, y') = \begin{cases} \frac{1}{2\sigma}(y - y')^2 & \text{if } |y - y'| \leq \sigma \\ |y - y'| - \frac{\sigma}{2} & \text{if } |y - y'| > \sigma \end{cases} $$
    在 PyTorch 中，可以直接使用 `torch.nn.HuberLoss` 来实现，其中 `delta` 参数对应于公式中的 $\sigma$：
    ```python
    import torch.nn as nn
    # 假设 delta (sigma) 的值为 1.0
    huber_loss = nn.HuberLoss(delta=1.0)
    ```
    `HuberLoss` 在 $|y - y'| \leq \delta$ 时使用二次方损失，在 $|y - y'| > \delta$ 时使用线性损失，这使其对异常值比 MSE 更加鲁棒，同时保持了在零附近的平滑性。

3. 如何访问线性回归的梯度

在 PyTorch 中，当你对一个计算图执行 `.backward()` 操作后，模型的参数（如权重和偏置）的梯度会被自动计算并存储在各自的 `.grad` 属性中。

对于线性回归模型中的第一个线性层 `net[0]`：

*   **访问权重梯度:**
    ```python
    gradient_w = net[0].weight.grad
    ```
*   **访问偏置梯度:**
    ```python
    gradient_b = net[0].bias.grad
    ```

`gradient_w` 和 `gradient_b` 将是 `torch.Tensor` 对象，包含了模型参数关于损失函数的一阶导数。这些梯度值在优化器的 `step()` 方法中被用来更新模型参数。